In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
import timeit
import scipy.sparse.linalg as spsplin
import scipy.linalg as splin
import time
# import networkx as nx
# from matplotlib.animation import FuncAnimation
from scipy.sparse import diags, csr_matrix
from scipy.sparse.linalg import spsolve
from IPython.display import HTML
from tqdm import tqdm
%matplotlib inline

In [ ]:
def loss_function(x, b):
    # Placeholder for the loss function
    return np.sum((x - b) ** 2)

def regularization_g(x):
    # Placeholder for regularization function g
    return np.sum(np.abs(x))

def regularization_h(z):
    # Placeholder for regularization function h
    return np.sum(np.square(z))





def nystrom_approximation(A, s):
    # Placeholder for the Nyström approximation function
    # This should return U and A based on the Nyström method
    # For demonstration, we will return random matrices
    n = A.shape[0]
    indices = np.random.choice(n, s, replace=False)
    U = A[indices, :]
    return U, A

def nystrom_pcg(A, b, tol):
    # Placeholder for the Nyström PCG method
    # This should return the solution to the linear system Ax = b
    # For demonstration, we will return a random vector
    return np.random.rand(A.shape[1])


'''
input feature matrix A, response b
    , loss function l, regularization g and h
    , stepsize p, positive summable sequence \epsilon_{k}
'''


def nysadmm(A, b, loss_func, reg_g, reg_h, p, epsilon_sequence, summable_sequence):
    # Initialize variables
    k = 0
    u_k = np.zeros(A.shape[1])  # Initialize u_k
    x_k = np.zeros(A.shape[1])  # Initialize x_k
    # z_k = np.zeros(A.shape[1])  # Initialize z_k

    # Compute the Nyström approximation
    U, Lambda = nystrom_approximation(A.T @ H @ A + H^g, summable_sequence)
    # U, A_nystrom = nystrom_approximation(A.T @ A + np.eye(A.shape[1]), summable_sequence)

    while True:#condition for converges x_k< smth
        # Step 1: Solve for x_k+1 using Nyström PCG
        x_k = nystrom_pcg(A_nystrom, b, epsilon_sequence[k])

        # Step 2: Update z_k+1
        z_k = np.argmin(lambda z: reg_h(z) + p/2*np.linalg.norm(x_k - z + u_k)**2)

        # Step 3: Update u_k+1
        u_k = u_k + x_k - z_k

        # Check for convergence (you can define your own convergence criteria)
        if np.linalg.norm(x_k_plus_1 - x_k) < epsilon_sequence[k]:
            break

        # Update variables for the next iteration
        x_k = x_k_plus_1
        z_k = z_k_plus_1
        u_k = u_k_plus_1
        k += 1

    return x_k  # Return the solution

# Example usage
A = np.random.rand(100, 50)  # Feature matrix
b = np.random.rand(100)       # Response vector
p = 0.1                       # Stepsize
epsilon_sequence = np.linspace(1e-3, 1e-6, 100)  # Tolerance sequence
s = 10                        # Number of samples for Nyström approximation

solution = nysadmm(A, b, loss_function, regularization_g, regularization_h, p, epsilon_sequence, s)
print("Solution:", solution)

IndexError: index 100 is out of bounds for axis 0 with size 100

In [ ]:
import numpy as np

def f(x):
    """Example function f(x) = 0.5 * ||x||^2"""
    return 0.5 * np.dot(x, x)

def g(z):
    """Example function g(z) = ||z - 1||^2"""
    return np.sum((z - 1) ** 2)

def grad_f(x):
    """Gradient of f(x)"""
    return x  # Gradient of 0.5 * ||x||^2 is x

def prox_g(z, lam):
    """Proximal operator of g(z)"""
    return np.clip(z, 1 - lam, 1 + lam)  # Projection onto the interval [1-lam, 1+lam]

def admm(A, B, c, rho, num_iterations):
    """ADMM algorithm implementation"""
    m, n = A.shape
    x = np.zeros(n)
    z = np.zeros(m)
    u = np.zeros(m)  # Dual variable

    for k in range(num_iterations):
        # x-update
        x = np.linalg.solve(np.eye(n) + rho * A.T @ A, rho * A.T @ (c - B @ z + u))

        # z-update
        z_old = z.copy()
        z = prox_g(c - A @ x + u, 1/rho)

        # u-update
        u += A @ x + B @ z - c

        # Optional: Print the objective value
        objective_value = f(x) + g(z)
        print(f"Iteration {k+1}, Objective Value: {objective_value}")

    return x, z

# Example usage
if __name__ == "__main__":
    # Define problem parameters
    A = np.array([[1, 2], [3, 4]])
    B = np.array([[1, 0], [0, 1]])
    c = np.array([1, 1])
    rho = 1.0
    num_iterations = 100

    x_opt, z_opt = admm(A, B, c, rho, num_iterations)
    print("Optimal x:", x_opt)
    print("Optimal z:", z_opt)

Iteration 1, Objective Value: 1.3673469387755102
Iteration 2, Objective Value: 1.2494960433152853
Iteration 3, Objective Value: 1.1651756870011645
Iteration 4, Objective Value: 1.104753462827945
Iteration 5, Objective Value: 1.1449343230002709
Iteration 6, Objective Value: 1.3341837976966244
Iteration 7, Objective Value: 1.452145665197893
Iteration 8, Objective Value: 3.4739272106542582
Iteration 9, Objective Value: 7.307175956025622
Iteration 10, Objective Value: 22.42145709947075
Iteration 11, Objective Value: 79.9865393959946
Iteration 12, Objective Value: 301.3475331942473
Iteration 13, Objective Value: 1156.4477618787655
Iteration 14, Objective Value: 4465.213963280708
Iteration 15, Objective Value: 17274.166348947747
Iteration 16, Objective Value: 66871.38461440305
Iteration 17, Objective Value: 258919.41897184434
Iteration 18, Objective Value: 1002646.4504897873
Iteration 19, Objective Value: 3882259.3396983976
Iteration 20, Objective Value: 15031169.888139497
Iteration 21, Obje

In [ ]:
import numpy as np

def randomized_nystrom(A, num_samples, num_components):
    """
    Randomized Nyström approximation.

    Parameters:
    A : np.ndarray
        The input matrix (n x n).
    num_samples : int
        The number of samples to take from the matrix.
    num_components : int
        The number of components to keep in the approximation.

    Returns:
    np.ndarray
        The approximated matrix.
    """
    n = A.shape[0]

    # Step 1: Randomly sample columns from A
    indices = np.random.choice(n, num_samples, replace=False)
    S = A[:, indices]  # Sampled columns

    # Step 2: Compute the submatrix
    C = S.T @ S  # C is the (num_samples x num_samples) matrix

    # Step 3: Compute the eigenvalues and eigenvectors of C
    eigvals, eigvecs = np.linalg.eigh(C)

    # Step 4: Select the top num_components eigenvectors
    idx = np.argsort(eigvals)[-num_components:]  # Indices of the largest eigenvalues
    W = eigvecs[:, idx]  # Top eigenvectors

    # Step 5: Compute the approximate eigenvalues
    Lambda = np.diag(np.sqrt(eigvals[idx]))

    # Step 6: Compute the final approximation
    B = S @ W @ np.linalg.inv(Lambda)  # Approximation of the original matrix

    return B

# Пример использования
if __name__ == "__main__":
    # Создаем случайную симметричную матрицу
    n = 100
    A = np.random.rand(n, n)
    A = (A + A.T) / 2  # Симметризация
    print(A)

    # Параметры
    num_samples = 20
    num_components = 10

    # Получаем приближенную матрицу
    B = randomized_nystrom(A, num_samples, num_components)

    print("Приближенная матрица:")
    print(B)

[[0.05306471 0.77271686 0.42562513 ... 0.74378368 0.57803667 0.48008357]
 [0.77271686 0.76821154 0.69099572 ... 0.49756584 0.94173682 0.27029396]
 [0.42562513 0.69099572 0.92320297 ... 0.25087498 0.38165381 0.51318503]
 ...
 [0.74378368 0.49756584 0.25087498 ... 0.01276707 0.76517296 0.4657315 ]
 [0.57803667 0.94173682 0.38165381 ... 0.76517296 0.54238506 0.48537651]
 [0.48008357 0.27029396 0.51318503 ... 0.4657315  0.48537651 0.17346481]]
Приближенная матрица:
[[-1.45483027e-01 -8.72239781e-02 -3.06116237e-02 -2.88756069e-02
  -1.93512923e-01 -7.64441810e-02 -1.51092884e-02  1.46413881e-01
   1.80907387e-01  9.99909172e-02]
 [ 1.37665346e-02  6.92168706e-02  1.25702149e-01 -2.51350604e-01
   1.26472605e-02 -2.06548141e-01 -1.07099556e-01 -1.43996146e-02
   1.01170283e-01  1.10712801e-01]
 [ 5.05508525e-02 -2.00126841e-03 -8.85194814e-02  2.25957162e-01
  -2.25420285e-02 -9.55737185e-03 -2.08705419e-01  1.82243151e-01
  -2.39752987e-02  9.92459520e-02]
 [ 4.98018255e-02 -4.19607509e-02

In [ ]:
print(np.linalg.norm(A, ord=2))



50.504577121818


In [ ]:
print(np.linalg.norm(B, ord=2))


1.000000000000002


In [ ]:
import numpy as np
from scipy.linalg import cholesky, svd, solve_triangular
from sklearn.datasets import make_spd_matrix


def nystrom_approximation(H, s):
    """
    Computes the Nyström approximation of a PSD matrix H using a randomized algorithm.

    Parameters:
    H (ndarray): A positive semi-definite matrix of shape (d, d).
    s (int): The sketch size.

    Returns:
    H_nys (ndarray): The Nyström approximation of the matrix H.
    """
    d = H.shape[0]

    # Step 1: Generate Gaussian test matrix
    # Omega = np.random.randn(d, s)
    Omega = np.random.normal(0, 1, (d, s))
    # Omega = np.random.normal(0, 1, (d, d))

    # Step 2: Thin QR decomposition
    Q, _ = np.linalg.qr(Omega, mode='reduced')

    # Step 3: Matrix-vector products
    Y = H @ Q  # H * Omega
    # Y = H @ Omega

    # Step 4: Compute shift
    # print(np.linalg.norm(Y, 2))
    # nu = np.finfo(float).eps * np.linalg.norm(Y, 2)
    nu = 1/(np.linalg.norm(Y, 2)+1)
    print(nu)

    # Step 5: Add shift for stability
    Y_nu = Y + nu * Q

    # Step 6: Cholesky decomposition
    C = cholesky(Q.T @ Y_nu, lower=True)

    print('C_size', C.shape)
    print('Y_nu size', Y_nu.shape)
    print()

    # Step 7: Triangular solve
    # B = np.linalg.solve(C, Y_nu)
    # B = solve_triangular(C, Y_nu)
    # B = solve_triangular(Y_nu, C)
    C_inv = np.linalg.inv(C)
    # print(C_inv)
    print('inverse C shape', C_inv.shape())
    B = Y_nu @ C_inv
    print(B)
    # print('B shape', B.shape())

    # Step 8: Thin SVD
    U, Sigma, _ = svd(B, full_matrices=False)
    print(U.shape)


    # Step 9: Remove shift and compute eigenvalues
    # Id = np.eye(s)
    Id = np.ones(Sigma.shape)
    # print(Id)
    print('sigma', Sigma)
    lambda_prime = np.maximum(0, Sigma**2 - nu*Id)

    # print(lambda_prime)
    # Step 10: Nyström approximation
    H_nys = U @ np.diag(lambda_prime) @ U.T

    return H_nys

# Example usage:
matrix = make_spd_matrix(100)
H = matrix
H_nys = nystrom_approximation(H, s=10)

0.03822156518890055
C_size (10, 10)
Y_nu size (100, 10)



TypeError: 'tuple' object is not callable